# Modeling the potential of CRISPR gene drives for rodent management
<hr style="border:2px solid gray"> </hr>
This notebook accompanies "Modeling the potential of CRISPR gene drives for rodent management", and can be used to perform additional sensitivity analyses on subspaces of the gaussian process models, make additional two-at-a-time plots, and three-at-a-time animated plots.

First, some imports:

In [ ]:
from rat_gp import *
from rat_plot import *
from IPython.display import HTML
%matplotlib inline

<hr style="border:1px solid gray"> </hr>

## 1. Loading a model
First, load the model and check model predictions are against the default testing set via the check_accuracy function. A different test set can be specified as a parameter of the check_accuracy function.

The load_model function has three arguments: model_name, model_type, and force_cpu.

Five models are available to specify via model_name:
- "female_fertility_homing"
- "female_fertility_homing_resistance"
- "viability_homing"
- "viability_homing_resistance"
- "y_shredder"

For each of these, two model_type settings are available:
- "composite"
- "suppression_rate"

The "suppression_rate" models are slightly more accurate, but have much looser confidence bounds, which may make the "composite" model more suitable for plotting purposes.

The code will automatically detect whether or not a CUDA enabled GPU is available to accelerate the model. To prevent the model from using your GPU if you have one, add "force_cpu=True" to the function call. There is no reason to do this unless you're doing some other computing work with your GPU or if you get a GPU out of memory error.

Note: Gaussian processes are stochastic. Loading the model requires that the model undergo a single training iteration on the test data, which is done internally by the function. This means that the loading function is also stochastic, and the model can vary from load to load by a very small amount.

In [ ]:
# Choose a model name from:
    # "viability_homing", 
    # "viability_homing_resistance",
    # "female_fertility_homing",
    # "female_fertility_homing_resistance",
    # "y_shredder".
# Choose a model mode from: "composite", "suppression_rate".

model = load_model(model_name="female_fertility_homing", model_type="composite", force_cpu=False)
model.check_accuracy()

# Note: this can take a minute or two on some computers.

<hr style="border:1px solid gray"> </hr>

## 2. Two-at-a-time analyses

Using the model to generate heatmap plots is straight forward, and is done using the plot_2d function, which has the following signiture:

plot_2d(model, x_param, y_param, fixed_params, param_ranges, x_and_y_steps)

- model: The name of the GP model to plot from. This is just "model" if you're using the code above.
- x_param: name of the parameter to plot on the x-axis.
- y_param: name of the parameter to plot on the y-axis.
- fixed_params: a dictionary object containing the fixed values of the other parameters. If the x and y parameters being plotted are in this dictionary, they will be ignored. If the dictionary is missing parameters, the default parameter values will be used for those parameters. This argument can be ommited, in which case default parameters are used.
- param_ranges: a dictionary of ranges through which to vary parameters if they are selected as x and y parameters. This argument can be ommited, in which case the full standard parameter ranges are used.
- x_and_y_steps: the number of steps the x and y parameters will be stepped through. The plots in the the manuscript used 1000 for this value, but this may take a while on a computer that doesn't have a fast GPU.

The parameter names that can be used for x_param and y_param are the same as the names in the fixed_params and param_ranges dictionary objects below.

In [ ]:
# These are the default fixed parameters for the model.
# Changing these values will change the fixed parameters (which are not shown in the plot):
fixed_params = {
        'Density': 1000.0,
        'Island side length': 2.0,
        'Interaction distance': 75.0,
        'Avg. dispersal': 250.0,
        'Monthly survival rate': 0.9,
        'Litter size': 4.0,
        'Itinerant frequency': 0.1,
        'Itin. dispersal multiplier': 2.0,
        'Release Percentage': 0.1,
        'Drive fitness': 1.0,
        'Drive efficiency': 0.9,
        'Resistance rate': 0.0,
        'R1 rate': 0.0
}

# These are the standard parameter ranges, with the exception that the minimum levels for
# "Drive efficiency" and "Resistance rate" can be decreased to 0.5 in the models for
# the homing drives without resistance.
# Changing the ranges of the parameters being plotted will change the range of the plot:
param_ranges = {
        "Density" : (600, 1500),
        "Island side length" : (1, 5),
        "Interaction distance" : (60, 300),
        "Avg. dispersal" : (25, 1000),
        "Monthly survival rate" : (0.7, 0.95),
        "Litter size" : (2, 8),
        "Itinerant frequency" : (0, 0.5),
        "Itin. dispersal multiplier" : (1, 5),
        "Release Percentage" : (0.01, 0.5),
        "Drive fitness" : (0.75, 1.0),
        "Drive efficiency" :(0.75, 1.0),
        "Resistance rate" : (0.0, 0.1),
        "R1 rate" : (0.0, 0.02)
}
# Actually, you can specify fixed parameters and parameter ranges that are outside those
# used to train the models, but predictions will get stranger and confidence intervals will get
# wider the further the model tries to extrapolate beyond data it has seen.

# Plot 100 X 100 predictions of "Drive fitness" against "Drive efficiency"
# using the fixed parameters and parameter ranges specified above:
plot_2d(model, "Drive fitness", "Drive efficiency", fixed_params, param_ranges, x_and_y_steps=100)

<hr style="border:1px solid gray"> </hr>

## 3. Three-at-a-time analyses

The model can also be used generate animated heatmaps, showing how a 2d slice of the model changes in response to a third parameter. This is done with the animated_plot function, which has the following signiture:

animated_plot(model, x_param, y_param, time_param, fixed_params, param_ranges, x_and_y_steps, time_steps)

All arguments are the same as above, except two which have been added:

- z_param: the third parameter, which will be varied through the course of the animation.
- z_steps: the number of steps for the time parameter.

In [ ]:
# Fixed parameters and parameter ranges are specified the same way as thhe two-at-a-time plots.

# As before, fixed_params doesn't need to include any params that you
# want to keep at the default values, e.g. you could just change the
# litter size parameter and keep other fixed params at default:
fixed_params = {
        'Litter size': 2.5,
}

# As before, param_ranges doesn't need to include any params that aren't
# being varied, or that you want to keep at the standard ranges, e.g.
# to show a plot area with a specified range of drive fitness and efficiency:
param_ranges = {
        "Drive fitness" : (0.6, 1.0),
        "Drive efficiency" :(0.6, 1.0),
}

# This plot can take a while to generate depending on the number of
# x, y, and z steps, so adjust those values accordingly.
anim = animated_plot(model, "Drive fitness", "Drive efficiency", "Monthly survival rate",
                           fixed_params, param_ranges, x_and_y_steps=100, z_steps=20)

# To display the animation as an interactable javascript widget.
# This may have issues with some browsers.
HTML(anim.to_jshtml())

# # Alternatively, the animation can be displayed as an embeded video:
# # This also may have issues with some browsers, but hopefully one of these t!
# # This requires ffmpeg:
# HTML(anim.to_html5_video())

# # To save the animation using ffmpeg:
# anim.save('AnimatedPlot.mp4', writer=animation.writers['ffmpeg'](fps=12))

# Matplotlib likes to display the first frame of the animation as its own plot for some curious reason.
# There are ways to prevent this, but they seem not to work consistently accross browsers, so its best to ignore it.

# Note: this can take a minute or two on some computers.
# If it is too slow, change around the number of x and y steps or the number of z steps.

<hr style="border:1px solid gray"> </hr>

## 4. Sensitivity analysis
It is also straight forward to coduct and plot a Sobol sensitivity analyis of the model.

The sensitivity_analysis function has three parameters:

- base_sample: Correlates to the number of model evaluations. We used 1000000 in the manuscript, but that might take a very long time for machines that don't have top end GPUs, and takes 45 minutes to an hour even with a high end system.
- param_ranges: an optional dictionary object containing parameter ranges, as for the plots above. Paramters that are not included in this dictionary will use standard paramter ranges for the model. If this argument is not supplied, the sensitivity analysis will use the entire standard parameter space.
- verbose: prints the analysis as text.

The sa_plot function plots the results of the sensitivity analysis. It has an optional argument of "limit_num_second_order_bars", which limits the number of items that will be plotted on the second order analysis to the n largest. This can be set to "limit_num_second_order_bars=None", in which case the plot will be very long.

In [ ]:
# These are the default parameter ranges:
# param_ranges = {
#         "Density" : (600, 1500),
#         "Island side length" : (1, 5),
#         "Interaction distance" : (60, 300),
#         "Avg. dispersal" : (25, 1000),
#         "Monthly survival rate" : (0.7, 0.95),
#         "Litter size" : (2, 8),
#         "Itinerant frequency" : (0, 0.5),
#         "Itin. dispersal multiplier" : (1, 5),
#         "Release Percentage" : (0.01, 0.5),
#         "Drive fitness" : (0.75, 1.0),
#         "Drive efficiency" :(0.75, 1.0),
#         "Resistance rate" : (0.0, 0.1),
#         "R1 rate" : (0.0, 0.02)
# }

# Let's try a sensitivity analysis with a interaction distance dispersal distance both fixed at a minimal level,
# but other paramters at their standard ranges.
# These can be set to single numbers, instead of a pairs, unlike in the plots above:
param_ranges = {"Interaction distance" : 60, 
                "Avg. dispersal" : 25}

sa = model.sensitivity_analysis(base_sample=1000, param_ranges=param_ranges, verbose=False)
sa_plot(sa, limit_num_second_order_bars=12)